In [38]:
import pandas as pd
import numpy as np
import hojaVida as hv
#"C:\\Users\\CO-166\\Desktop\\AdquisInfo\\"
rutaPrincipal="C:\\Users\\jnat_\\Desktop\\labdatos\\AdquisInfo\\"

In [41]:
hv.SmartSearch.process_database(ruta=rutaPrincipal+"baseHV.csv", ruta_out=rutaPrincipal+"outHV.csv")


ParserError: Error tokenizing data. C error: Expected 137 fields in line 9683, saw 177


In [79]:
#base de datos aplicar conv
# rutaAC = "C:\\Users\\jnat_\\Desktop\\labdatos\\AdquisInfo\\ba.csv"
rutaAC = 'C:/Users/jnat_/Desktop/labdatos/AdquisInfo/ba.csv'
tablaAC = pd.read_csv(rutaAC, on_bad_lines='skip')
#base de datos hoja de vida
rutaHV="C:/Users/jnat_/Desktop/labdatos/AdquisInfo/outHV.csv"
tablaHV=pd.read_csv(rutaHV,encoding='latin1', on_bad_lines='skip', lineterminator='\n')

C:\Users\jnat_\AppData\Local\Temp\ipykernel_4984\4161750375.py:4: DtypeWarning: Columns (11,13,14,16,18,19,20,21,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  tablaAC = pd.read_csv(rutaAC, on_bad_lines='skip')
C:\Users\jnat_\AppData\Local\Temp\ipykernel_4984\4161750375.py:7: DtypeWarning: Columns (0,6,10,13,18,19,27,28,29,32,33,34,35,36,37,38,39,40,42,43,63,64,65,67,68,73,74,75,76,80,82,87,96,110,120,122,127,128,129,130,131,132,133,138,139,142,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  tablaHV=pd.read_csv(rutaHV,encoding='latin1', on_bad_lines='skip', lineterminator='\n')


In [81]:
def normalize(s):
    #"""Función para remplazar una lista de caracteres por otra"""
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        (" ", "_"),
        ("ñ","n"),
        ("(", ""),
        (")", ""),
        (",", ""),
        (".", "")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.lower(), b.lower()).lower()
    return s


def normalizar_titulos(df):
    #"""Esta función recibe un dataframe, tranforma los titulos y devuelve el dataframe completo con los nuevos titulso normalizados """
    column_names = []
    for x in df.columns:
        y = normalize(x)
        column_names.append(y)
    df.columns = column_names
    return df

In [82]:
#func para para pasar nivel edu a número
def toNumbNivelEdu(df,nivEdu):
    df["nivelEducativo"] = np.where(
                (df[nivEdu] == "NINGUNO"),
                0,
                np.where(
                    (df[nivEdu] == "BASICA PRIMARIA"),
                    1,
                    np.where(
                        (df[nivEdu] == "MEDIA ACADEMICA (BACHILLER)"),
                        2,
                        np.where(
                            (df[nivEdu] == "TECNICA PROFESIONAL"),
                            3,
                            np.where(
                                (df[nivEdu] == "TECNOLOGICA"),
                                4, np.where(
                                    (df[nivEdu] == "PROFESIONAL"),
                                    5,
                                    np.where(
                                        (df[nivEdu] == "POSTGRADO"),
                                        6,
                                        0
                                    )
                                )
                            )
                        )
                    )
                )
            )
    return df


In [83]:
#para pasar a boleano las pruebas(tiene o no)
def to_bool_pruebas(tabla):
    lista = []
    lista2 = []
    lista3=[]
    listaGen=[]
    for x in tabla.index:
        if str(tabla["prueba_1"][x]).__contains__("."):
            lista.append(1)
        else:
            lista.append(0)

        if str(tabla["prueba_2"][x]).__contains__("."):
            lista2.append(1)
        else:
            lista2.append(0)

        if str(tabla["prueba_3"][x]).__contains__("."):
            lista3.append(1)
            
        else:
            lista3.append(0)



    dataFrame = pd.DataFrame(lista, columns=["PruebaCont_01"])
    dataFrame1 = pd.DataFrame(lista2, columns=["PruebaCont_02"])
    dataFrame2 = pd.DataFrame(lista3, columns=["PruebaCont_03"])
    dataGen = pd.DataFrame(listaGen, columns=["Genero"])

    return pd.concat([tabla, dataFrame,dataFrame1,dataFrame2, dataGen], axis=1)

In [84]:
#limpieza encabezados
tablaACF=normalizar_titulos(tablaAC)
tablaHVF=normalizar_titulos(tablaHV)
#se eliminan datos repetidon entre tablas
dfAC=tablaACF.drop(columns=['fecha_de_nacimiento','genero_de_nacimiento','estado_civil', 'tipo_de_documento','departamento_de_residencia','ciudad_de_residencia'])
#se iguala el nombre que se va a utilizar para unir las tablas
tablaHVF.rename(columns = {'no_identificacion':'documento'}, inplace = True)
#se pasa nivel educativo a una "calificación" numérica
dfAC=toNumbNivelEdu(dfAC,'nivel_educativo')
#se pasan a boleanos las pruebas
dfAC=to_bool_pruebas(dfAC)
#se eliminan las empresas de prueba
for x in dfAC.index:
    if dfAC["empresa"][x].__contains__("EMPRESA DE PRUEBA"):
        dfAC.drop(index=x, inplace=True)
dfAC


AttributeError: 'float' object has no attribute '__contains__'

In [37]:
#se unen las tablas 
tablaACHV=dfAC.merge(tablaHVF, on='documento', how='inner')

for x in tablaACHV.columns:
    print (x)

nombres_y_apellidos
nivel_educativo_x
empresa
sede
cargo
documento
funciones
rechazar_candidato
contratado_por
motivo_de_rechazo
numero_de_documento
estado_trabajador
pruebas_psicotecnicas
prueba_1
prueba_2
prueba_3
experiencia_-_conocimientos
habilidades
contratado
pruebas_psicotecnicas1
desarrollo_familiar_
tipo_de_proceso
added_time_x
added_user_x
actividad_economica
departamentosede
ciudadsede
nivelEducativo
PruebaCont_01
PruebaCont_02
PruebaCont_03
Genero
unnamed:_0
primer_apellido
segundo_apellido
primer_nombre
segundo_nombre
tipo_de_documento
nacionalidad
no_pasaporte
fecha_de_nacimiento
edad
genero_de_nacimiento
identidad_genero
orientacion_sexual
departamento_de_residencia
barrio
correo_electronico
telefono
celular
celular_alterno
added_time_y
added_user_y
hv_con_contrato_activo
departamento_de_expedicion
nivel_educativo_y
nucleo_familiar
referencias_personales
inicio
fin
minutos_trancurridos
modified_user
modified_time
hv_actualizada
no_de_hijos
politica_de_privacidad_y_trata

In [ ]:
# from cgitb import text
# import requests
# import json
# import pandas
# import urllib.parse
# import urllib3

# def token_():
#     parameters_= {
#         "ClientSecret":"fdaf31a7-d9b6-461b-bf6c-a834d807aa78",
#         "ClientId": "e962f0a3-c4ae-4d62-8663-123cd3615ce6"
#         }  
#     response_=requests.post("https://api.psicoweb.com/login", data = parameters_)
#     return json.loads(response_.text)["data"]["token"]

# autorizacion = {"Authorization": token_()}
# listaCar= []

# descargable = requests.get("https://api.psicoweb.com/v1/aplicaciones/link/ExcelConcentrado?prueba=Terman&puesto=NECNOCTerVal", headers = autorizacion)
# infoCargos= json.loads(descargable.text).get("data")
# linkdes=infoCargos.get("link")
# resp = requests.get(linkdes)
# archivo="C:\\Users\\CO-166\\Desktop\\pruebas combinaciones\\test.xlsx"

# open(archivo, "wb").write(resp.content)
# #df = pd.read_excel(archivo, header=6, usecols='B:z')

# print(linkdes)

https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOszRC0cKB5VtM4PYrwEqHVdR0fNhIMynhl2Rj7tjm1aolnZyQpXSIQmanCdJ2B1hb%2fJhl%2fl9A2fK0i%2bF2WHgeKdO%2faiiDw5RoeHyC5Ls1FIlVGwiUfCL4rZw%2f53iuZ0iwNgAWIKkMdd4cMaruP7PdoXg%3d


In [ ]:
# from pathlib import Path
# archivo="C:\\Users\\CO-166\\Desktop\\pruebas combinaciones\\test.xlsx"
# archivoModf= Path.cwd() / archivo
# tablaPsicoweb = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
# tablaPsicoweb

C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\1499190619.py:4: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  tablaPsicoweb = pd.read_excel(archivoModf, header=5, usecols='B:AZ')


,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,Información (I),Juicio (II),Vocabulario (III),Síntesis (IV),Concentración (V),Análisis (VI),Abstracción (VII),Planeación (VIII),Organización (IX),Atención (X),PN Total,CI,Interpretación,C. Aprendizaje,Compatibilidad
0,1010178224req14599,JOHN ROJAS VILLABON,Bachillerato,27,02/07/2022 09:12:07 AM,10,22,24,14,16,12,9,9,12,12,140,104,NORMAL,NORMAL,-
1,1023954285req14599,CARLOS - MOLANO,Bachillerato,26,09/07/2022 12:26:38 PM,15,22,28,10,16,14,19,13,16,20,173,114,SUPERIOR,SUPERIOR,-
2,1032503454req14599,JUAN - CASTRO,Bachillerato,23,02/07/2022 07:08:34 AM,15,22,22,13,8,16,12,9,12,6,135,103,NORMAL,NORMAL,-
3,1143149688req14599,JESUS AMAYA PERTUZ,Bachillerato,27,01/06/2022 05:24:05 PM,12,16,16,13,14,10,17,3,15,6,122,99,NORMAL,NORMAL,-


In [53]:
#obtiene las pruebas de psicoweb
from cgitb import text
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
from pathlib import Path

#se realiza la consulta del tocken para acceder a la info
def token_():
    parameters_= {
        "ClientSecret":"fdaf31a7-d9b6-461b-bf6c-a834d807aa78",
        "ClientId": "e962f0a3-c4ae-4d62-8663-123cd3615ce6"
        }  
    response_=requests.post("https://api.psicoweb.com/login", data = parameters_)
    return json.loads(response_.text)["data"]["token"]

autorizacion = {"Authorization": token_()}
#enlace para solicitud de cargos exixtentes
linkDescCargo = requests.get("https://api.psicoweb.com/v1/Puestos", headers = autorizacion)
infoCargos = json.loads(linkDescCargo.text).get("data")
#se crea una lista vacía para almacenar los cargos que se obtengan
listaCar= []

#pruebas que psicoweb no entrega en excel
#"NEC","NIC","NOC",,"PVC"
#lista de las pruebas que se van a buscar
listaPrubas=["Cleaver","Valores","FP16","Terman","Zavic"]

#para cda cargo obtenido se toma el nombre y se pasa a formato utf-8 para crear la consuta por medio del enlace
for x in infoCargos:
    var=x["nombre"]
    #se agregan los cargos agregados a la lista
    listaCar.append(urllib.parse.quote(var.encode('utf-8')))

#para cada tipo de prueba hace las combinaciones con todos los cargos obtenidos para descargar
for a in listaPrubas:
    count=0
    for y in listaCar:
        try:
            linkDesc = requests.get("https://api.psicoweb.com/v1/aplicaciones/link/ExcelConcentrado?prueba="+a+"&puesto="+y, headers = autorizacion)
            infoLinkDesc= json.loads(linkDesc.text).get("data")
            linkdes=infoLinkDesc.get("link")
            resp = requests.get(linkdes)
            #se crea ruta en la que se va a almacenar el archivo de forma temporal para ir añadiendp la tabña por prueba
            archivoTemporal=rutaPrincipal+"temporal.xlsx"
            open(archivoTemporal, "wb").write(resp.content)
            #
            archivoModf= Path.cwd() / archivoTemporal
            #valida la prueba que está tomando para crear o añadir a la tabla de la prueba específica
            if a == "FP16":
                if count== 0:
                    try:    
                        pruebaFP16 = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        #contador para identificar si es el primer proceso para crear la tabla o añadir a la tabla de la prueba existente
                        count=count+1
                    #si no corre lo deja pasar porque es posible que esa combinación de prueba y cargo no exista    
                    except Exception as e:    
                        print (e)
                else:
                    
                    try:
                        #si ya lo creó entonces añade a la tabla de la prueba princiapal
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaFP16 = pd.concat([pruebaFP16, df])
                    except Exception as e:    
                        print (e) 
            elif a == "Cleaver":
                if count== 0:
                    try:    
                        pruebaCleaver = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except Exception as e:    
                        print (e)
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaCleaver = pd.concat([pruebaCleaver, df])
                    except Exception as e:    
                        print (e)
            elif a == "PVC":
                if count== 0:

                    try:    
                        pruebaPVC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaPVC = pd.concat([pruebaPVC, df])
                    except:
                        pass
                    
            
            elif a == "Valores":
                if count== 0:

                    try:    
                        pruebaValores = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaValores = pd.concat([pruebaValores, df])
                    except:
                        pass

            elif a == "NEC":
                if count== 0:

                    try:    
                        pruebaNEC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNEC = pd.concat([pruebaNEC, df])
                    except:
                        pass
            elif a == "NIC":
                if count== 0:

                    try:    
                        pruebaNIC= pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNIC = pd.concat([pruebaNIC, df])
                    except:
                        pass

            elif a == "NOC":
                if count== 0:

                    try:    
                        pruebaNOC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNOC = pd.concat([pruebaNOC, df])
                    except:
                        pass     

            elif a == "Terman":
                if count== 0:

                    try:    
                        pruebaTerman = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaTerman = pd.concat([pruebaTerman, df])
                    except:
                        pass
                    
            elif a == "Zavic":
                if count== 0:

                    try:    
                        pruebaZavic = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        count=count+1
                    except:    
                        pass
                else:
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaZavic = pd.concat([pruebaZavic, df])
                    except:
                        pass                      
            print(linkdes)
        except Exception as e:    
            print (e)
        

C:\Users\CO-166\AppData\Local\Temp\ipykernel_46844\1953672.py:72: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  pruebaCleaver = pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs%2bbPbkqXGWwCqCocZQujt8OKUAUUtEayenRvaKo4MWCB6AmHj4JHLARHA4ogPAIeEyvOpCIvT%2f%2bS3ma8uj0NyIf%2bGzTtHaKm7PeUgUABB3lVCipccFIQfnjl%2fhRjpnDX6OGSsan6bsAmhZCfmOW5qSw%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs4InRp70iX12d2JL5Q2b6VcyhnN4p2JYnn4nl83HlNcB7eC%2bSRwHb2TXmb%2bhAQ194bCUfoZTyl7P8vUCPfXVH52FVprJ9c06dHOhyX6DcLdbhxy4l3Yq0zzt6eHqUE%2bUxQvHkjSdw4gNUlIHBtLRhzg%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOwCMZTnAnUDGB5%2f70s6OxBK%2fxv1JjpNq%2fey8xzlShIfAr1lnnVlznVXXfKhjRwnPNd%2fnN1yBpPIkaaNkklv5PWla3x0KL9Z3mEfCgXeQHyfvh7WIGLIdo3HnAkdwUjYZOM%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOzpof1S5IIUUAfKMG7gD67i

C:\Users\CO-166\AppData\Local\Temp\ipykernel_46844\1953672.py:79: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs7EYKtFsKOXnH1vrG4wqH2rWDogDtTBwcyRZfemILnY0OLu2wzRPwFY7qlymWV4lfmphhhO7uv7h%2fKtL%2fyvgK0gMZlHfsD%2f4XtXZdoCeqSrnFM1E2MuXyYclL5GRBQnJs48x%2bBd2Yt%2fY5dp64twF2c4%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOypSlh1QC0%2bVSWlKUyeSh16u6q4yr1JLHMVWvK5ic6wKwPgQ38rvOUcksC1vHFlkMjc6m8JSaVcBJXDZqgO1tzb1h6EdzuveIQs2m%2fFJuLbOmu6Wv1kf8oZ67FjkCs1EGWDsLUZ9kLPv8EvNCoCh2La
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOyBVzKOy4UK7fwbKXq1QyPWk07%2flpMe4bgJIcKt7kWzGzHDEgLISdqF7eGoJ669cXlOIQIvRa8ooxIbWRiLJFFkyQlg9fjXdtOZXxywd4W6SS5XEKc%2bh8h2i8Fb4uiJj%2fQ%3d


C:\Users\CO-166\AppData\Local\Temp\ipykernel_46844\1953672.py:79: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


...

https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpbE1mm%2bC2qP%2fsH1sZ0dksIVwccvOaSfAhmKancmHnR4VSVZXQdoAoG8v0r%2f49EIxZ2O7UxGx9pi1XqNMxZqfiI7oGqPuzYUfrCxzX4ZGtawRX2v%2fy0LEepIPskis9X9%2bsL8K%2feVLeGlRnF%2fBVsc1cEs
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpZJ0okI%2bBjL2uga1%2b31Dy83jivFN60%2bdiR6993AApOwjKvhsG1X2ZQhiuRP1rcqm%2bvEp0nqH3ETJawzbM%2frCXxiW9rulfMaPHnnY1aflHohHAUw8oyyB3o9OAtmghV%2fAwq5EKNivY%2bs3K9Z4raShBQTyAVN5YLcgKkwkYjMVbSEdA%3d%3d
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpaECpK4SPUjAi2oiAlPkOGt92%2bWaR8iUkPgOu9eVkUkwo2v05PfhuZ7qclvGSFxm7rjEcUlBoc0tjzzBrT1kgw4uC8eRc0qaMiCtmaiAnCG5R3Bm5e1f2KgQx%2fGSYLNr9Pnx%2bVmVU%2b6E5Jf%2fLA%2b2Yl5
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpYkT4QZVJVGw9bH1ziCv7ueDBfGBG7XNKBVXdF%2bYJ0MXwr%2fY%2bUxv3h4Fk%2fl0m4pRX5dbfhHc9s%2fqdfEe4GS69D1sEzqJNgoTs8GNd6kWwqKiSAUptQikIPMwF1SXaGwTETIrctsUqG

C:\Users\CO-166\AppData\Local\Temp\ipykernel_46844\1953672.py:183: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR1hbR3%2fBU6Z%2btdYTcIZ6UYL0WzzBi7SRp%2bi0Dw6ldIKIAUIsp93b846OeZYQwEp9FuifX3WyTY9lN5JgPVjy4o4ZArdc2Ev%2f0m7YffrBFe%2f1SCCG0sSsMqMgxHfFZrN62DelFeIlM%2f3DqzIG0BBZso8
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR2hUcD9xPNDN6ZZeKR0ZPx858yXBnQ245BOupeFI%2b5v1gwJ0UH2tRmVGEB%2fNOV6s0HmYVWOlyZsgZd1hHDvLs1zRK%2btKTL%2bYY1A1GAqqpewoAkwiS9p07rQGDs2OE6HhlCKKEBn2SeLvCwsFrYNeLCG
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR3MdoluZYgUoFOE%2f9oRBi7du%2bKThJFXD5DPM3Tp1egz7N4cgHcgsAHC8Chq8porW8b5gstZyaSFfp6%2bfrpPhFAx%2fflrDKeyOtwo7Ny%2bKFqZWSS%2fmWc%2bNeaueUrjJneb0FYGIVeNl8aEe25xyxy6exTcU0UZwMAkwQuChpiNm6OTDQ%3d%3d
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR1UOfTB0aAI20WKKA4VOb02rJDXRjG14RjJS45mHnaACXN1Usoriif38Zck%2fMfwmHesN5tYg7twMyj0lO59X4rMToU%2brnkbnHITTMYdc7oS%2bhVSujjzOrkxPrQZnHFsKeJJqtc76I8n4TFQCmaMZ

C:\Users\CO-166\AppData\Local\Temp\ipykernel_46844\1953672.py:183: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR3%2bbhPTbYR5MbUtfHEexP7Nx%2bdmH4SFON9OhiW%2fWeT9WHLjnZtJ3E8htrRUzL8YAjOmW7YqvFT1k6XTtEJE6M8Gk%2fKYI6PhQzoQuZDD%2fUF6gd4nnjgatL%2bab3H2rhCc%2fV6n9XWaK9Av1xfJPzDd1tVnHFyEweoH47SajuM%2bRBvCeA%3d%3d
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR3%2bbhPTbYR5MbUtfHEexP7NLnoCDKngE3xIRiGTH%2fxA73E2upBfeVn987UinJ8NjACqOfbKL3kH%2fBshPt6pwomgcxlcr9OOd5NZDL7aLbjw6Sp0pwP9F%2fkg4RQJ3uCiW3H6HWW%2fZ7U%2fsmFW5gvekvMmSfwnf9cQiqmt4uX%2bOPfy7w%3d%3d
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR3%2bbhPTbYR5MbUtfHEexP7N8x3ZOmMiMcKf9A0aZxVyYj9mjXiYwh%2bLhN%2funxyqnxbYXFopbLNPCYfWQv9TOb78iSqN7s%2b9ZEryEv8wN%2b0xzC2a7QA9O5zzFnmxMcbR3GblFVoXboYAGSlDatpX7uqhilP0XJXPW8imXnBfk%2fQXPQ%3d%3d
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1CJCAcWPpchk%2fwC5OQ0mR2RaBYuHp%2bixGkg8Jb%2fQb7rCxXUK%2bOEn17WY%2bv8S73pTdaAugnOOWFWdMy58WrQIQg32F6%2f5mAZUwyPo

In [54]:
pruebaZavic.head(5)

,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,Moral (Mo),Legalidad (Le),Indiferencia (In),Corrupción (Co),Económico (Ec),Político (Po),Social (So),Religioso (Re)
0,72279827,ALEX OLIVO BARRETO,Profesional,39,30/08/2021 07:36:02 PM,31%,34%,18%,17%,28%,21%,35%,16%
0,1000329755,yuli dayanna Ramírez Aguillón,Técnico,22,08/03/2021 05:55:33 PM,33%,35%,17%,15%,26%,27%,33%,14%
1,1002538681,Paola Andrea Muñoz Jimenez,Tecnólogo,25,30/10/2020 04:21:38 PM,33%,32%,16%,19%,29%,24%,33%,14%
2,1012425259,Bryan David Muñoz Sarmiento,Tecnólogo,27,23/01/2021 11:26:48 AM,34%,32%,15%,19%,30%,24%,34%,12%
0,1116253565,Jairo Ivan Trujillo Valencia,Tecnólogo,30,19/08/2021 09:21:33 AM,31%,36%,15%,18%,25%,18%,35%,22%


In [55]:
pruebaValores

,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,TC,UE,EA,SH,PD,MR,TC.1,UE.1,EA.1,SH.1,PD.1,MR.1,Compatibilidad
0,1000787201,karen melissa bareño gutierrez,Profesional,30,10/11/2021 06:47:11 AM,83,70,40,50,58,48,-,-,-,-,-,-,-
0,1001278261,Franklin Ramos Gil,Bachillerato,21,08/02/2022 09:13:18 AM,92,32,58,53,53,62,-,-,-,-,-,-,-
1,1012457111,Laura rocio Perez Perez,Secundaria,24,08/11/2022 01:44:32 PM,70,70,63,38,52,57,-,-,-,-,-,-,-
2,1016081800,Brandon stid Bastidas Gil,Técnico,26,11/11/2021 06:05:57 AM,52,63,60,65,47,63,-,-,-,-,-,-,-
3,1022948763,Henry gomez Hernández,Técnico,33,11/11/2021 02:38:25 PM,68,83,43,50,55,50,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1051211958,Yenny Paola Sánchez Contreras,Bachillerato,26,14/12/2021 11:00:20 AM,78,53,48,65,43,62,-,-,-,-,-,-,-
2,1067859508,EDGAR ALTAMIRANDA PEREZ,Bachillerato,36,25/03/2021 11:41:36 AM,85,57,25,57,62,65,-,-,-,-,-,-,-
3,1069504287,Ender jose Gonzalez Ojeda,Bachillerato,43,25/03/2021 11:22:46 AM,72,47,58,67,38,68,-,-,-,-,-,-,-
4,1094916707,Andrés Torres Betancourt,Bachillerato,31,25/03/2021 09:30:56 AM,72,48,55,60,53,62,-,-,-,-,-,-,-


In [60]:
pruebaFP16

,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,PD,Esten,PD.1,Esten.1,PD.2,...,Esten.12,PD.13,Esten.13,PD.14,Esten.14,PD.15,Esten.15,PD.16,Esten.16,Compatibilidad
0,1048284536,Elizabeth Narvaez Silva,Técnico,31,07/03/2022 07:37:18 PM,15,9,9,8,24,...,2,11,5,17,8,7,5,11,9,-
0,1098763527,Jesús Ernesto Carrascal Archila,Tecnólogo,27,27/12/2021 08:16:16 PM,12,6,11,10,25,...,3,6,3,19,10,4,4,14,10,-
0,1001172730,Brandon andrey Ravelo Muñoz,Técnico,22,24/06/2021 09:21:52 PM,16,9,7,6,26,...,1,8,4,16,7,2,2,11,9,-
1,1022392691,monica martinez prieto,Técnico,28,24/06/2021 07:11:50 PM,12,6,2,1,22,...,3,8,3,14,6,4,3,9,7,-
2,1074135921,Julieth Adriana Bueno Torres,Técnico,25,25/06/2021 06:22:25 PM,13,7,5,4,20,...,4,10,5,12,5,9,6,4,4,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,16896030,Juan Carlos Muñoz Mazuera,Técnico,39,27/11/2020 05:08:49 PM,11,6,8,7,10,...,1,8,4,17,8,12,8,11,9,-
0,1014268707,Juan Sebastian Castiblanco Arismendi,Bachillerato,27,07/09/2022 06:55:41 PM,9,5,4,3,23,...,2,12,6,18,9,8,6,11,9,-
1,1016090882,Nicolas Becerra Aldana,Tecnólogo,25,06/09/2022 10:16:23 PM,9,5,6,5,16,...,6,4,2,14,6,7,5,11,9,-
2,1024599147,Nicolas Andres Anaya Ramirez,Secundaria,23,10/09/2022 07:44:40 AM,12,6,10,10,19,...,4,11,6,11,4,11,7,4,4,-


In [61]:
listaPruebas = [pruebaZavic,pruebaTerman,pruebaValores,pruebaCleaver,pruebaFP16]
listaNombre  = ["prubaZavic","pruebaTerman","pruebaValores","pruebaCleaver","pruebaFP16"]
count = 0
for prueba in listaPruebas:
    prueba.to_csv(rutaPrincipal+ listaNombre[count]+ ".csv")
    count += 1


In [6]:
prueba_Zavic= pd.read_csv(rutaPrincipal+"prubaZavic.csv")
prueba_Terman= pd.read_csv(rutaPrincipal+"pruebaTerman.csv")
prueba_Valores= pd.read_csv(rutaPrincipal+"pruebaValores.csv")
prueba_Cleaver= pd.read_csv(rutaPrincipal+"pruebaCleaver.csv")
prueba_FP16= pd.read_csv(rutaPrincipal+"pruebaFP16.csv")


C:\Users\jnat_\AppData\Local\Temp\ipykernel_4984\1209844210.py:3: DtypeWarning: Columns (6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  prueba_Valores= pd.read_csv(rutaPrincipal+"pruebaValores.csv")


In [67]:
test_Zavic= normalizar_titulos(prueba_Zavic)
test_Terman= normalizar_titulos(prueba_Terman)
test_Valores= normalizar_titulos(prueba_Valores)
test_Cleaver=normalizar_titulos(prueba_Cleaver)
test_FP16=normalizar_titulos(prueba_FP16)


for x in tablaPsicoweb.index

In [10]:
listaDataTest=[test_Zavic,test_Terman,test_Valores,test_Cleaver,test_FP16]
for prueba in listaDataTest:
    
    for x in prueba.index:
        listaData=[]
        if prueba["cedula"][x].__contains__("req") :
            y=prueba["cedula"][x].split("req")
            listaData.append([y[0],y[1]])
        else:
            listaData.append([prueba["cedula"][x], "0"])
    df=pd.DataFrame(listaData,columns=["documento","requi"])

    prueba = pd.concat([prueba,df], axis=1)


In [68]:
#se separa documento de requi en la cedula (aci x que no sirvió el for :'v) -- 
# si se corre mas de una vez se duplica

listaData=[]
for x in test_Zavic.index:
    
    if test_Zavic["cedula"][x].__contains__("req") :
        y=test_Zavic["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([test_Zavic["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

test_Zavic = pd.concat([test_Zavic,df], axis=1)

listaData=[]
for x in test_Terman.index:
    
    if test_Terman["cedula"][x].__contains__("req") :
        y=test_Terman["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([test_Terman["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

test_Terman = pd.concat([test_Terman,df], axis=1)

listaData=[]
for x in test_Valores.index:
    
    if test_Valores["cedula"][x].__contains__("req") :
        y=test_Valores["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([test_Valores["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

test_Valores = pd.concat([test_Valores,df], axis=1)

listaData=[]
for x in test_Cleaver.index:
    
    if test_Cleaver["cedula"][x].__contains__("req") :
        y=test_Cleaver["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([test_Cleaver["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

test_Cleaver = pd.concat([test_Cleaver,df], axis=1)

listaData=[]
for x in test_FP16.index:
    
    if test_FP16["cedula"][x].__contains__("req") :
        y=test_FP16["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([test_FP16["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

test_FP16 = pd.concat([test_FP16,df], axis=1)


In [31]:
test_FP16

,unnamed:_0,cedula,nombre,escolaridad,edad,fecha_de_aplicacion,pd,esten,pd1,esten1,...,esten13,pd14,esten14,pd15,esten15,pd16,esten16,compatibilidad,documento,requi
0,0,1048284536,Elizabeth Narvaez Silva,Técnico,31,07/03/2022 07:37:18 PM,15,9,9,8,...,5,17,8,7,5,11,9,-,1048284536,0
1,0,1098763527,Jesús Ernesto Carrascal Archila,Tecnólogo,27,27/12/2021 08:16:16 PM,12,6,11,10,...,3,19,10,4,4,14,10,-,1098763527,0
2,0,1001172730,Brandon andrey Ravelo Muñoz,Técnico,22,24/06/2021 09:21:52 PM,16,9,7,6,...,4,16,7,2,2,11,9,-,1001172730,0
3,1,1022392691,monica martinez prieto,Técnico,28,24/06/2021 07:11:50 PM,12,6,2,1,...,3,14,6,4,3,9,7,-,1022392691,0
4,2,1074135921,Julieth Adriana Bueno Torres,Técnico,25,25/06/2021 06:22:25 PM,13,7,5,4,...,5,12,5,9,6,4,4,-,1074135921,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,1,16896030,Juan Carlos Muñoz Mazuera,Técnico,39,27/11/2020 05:08:49 PM,11,6,8,7,...,4,17,8,12,8,11,9,-,16896030,0
908,0,1014268707,Juan Sebastian Castiblanco Arismendi,Bachillerato,27,07/09/2022 06:55:41 PM,9,5,4,3,...,6,18,9,8,6,11,9,-,1014268707,0
909,1,1016090882,Nicolas Becerra Aldana,Tecnólogo,25,06/09/2022 10:16:23 PM,9,5,6,5,...,2,14,6,7,5,11,9,-,1016090882,0
910,2,1024599147,Nicolas Andres Anaya Ramirez,Secundaria,23,10/09/2022 07:44:40 AM,12,6,10,10,...,6,11,4,11,7,4,4,-,1024599147,0


In [69]:
test_Zavic=test_Zavic.drop(columns=['cedula','unnamed:_0','escolaridad','edad','nombre'])
test_Cleaver=test_Cleaver.drop(columns=['cedula','unnamed:_0','escolaridad','edad','nombre'])
test_FP16=test_FP16.drop(columns=['cedula','unnamed:_0','escolaridad','edad','nombre'])
test_Terman=test_Terman.drop(columns=['cedula','unnamed:_0','escolaridad','edad','nombre'])
test_Valores=test_Valores.drop(columns=['cedula','unnamed:_0','escolaridad','edad','nombre'])



,fecha_de_aplicacion_x,pd,esten,pd1,esten1,pd2,esten2,pd3,esten3,pd4,...,s1,c1,compatibilidad1,d2,i2,s2,c2,compatibilidad2,valida,requi
0,15/10/2020 12:06:43 PM,13,7,4,3,16,4,16,8,13,...,84,84,-,45,45,79,92,-,No,0
1,02/09/2020 03:36:02 PM,7,3,10,10,17,4,10,5,12,...,84,84,-,69,60,89,84,-,No,0
2,26/05/2022 09:44:49 AM,12,6,10,10,20,6,3,1,11,...,11,98,-,60,18,79,48,-,Sí,0
3,14/04/2021 10:33:15 PM,8,4,7,6,23,7,11,5,12,...,65,73,-,60,30,69,92,-,Sí,0
4,14/04/2021 06:05:58 PM,13,7,4,3,22,7,13,6,12,...,75,84,-,40,60,89,84,-,Sí,0
5,15/04/2021 10:42:13 AM,12,6,8,7,19,5,8,4,7,...,84,96,-,23,30,89,96,-,Sí,0
6,16/04/2021 11:12:41 AM,12,6,5,4,23,8,10,5,12,...,65,55,-,60,60,69,84,-,Sí,0
7,14/04/2021 06:55:45 PM,8,4,3,3,14,3,6,2,13,...,58,93,-,87,2,89,84,-,Sí,0


In [73]:
dfZaFP=test_FP16.merge(test_Zavic, on='documento', how='left').merge(test_Cleaver, on='documento', how='left').merge(test_Terman, on='documento', how='left').merge(test_Valores, on='documento', how='left')

dfZaFP.columns

C:\Users\jnat_\AppData\Local\Temp\ipykernel_4984\1213811609.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'requi_x', 'fecha_de_aplicacion_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfZaFP=test_FP16.merge(test_Zavic, on='documento', how='left').merge(test_Cleaver, on='documento', how='left').merge(test_Terman, on='documento', how='left').merge(test_Valores, on='documento', how='left')
C:\Users\jnat_\AppData\Local\Temp\ipykernel_4984\1213811609.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'compatibilidad_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfZaFP=test_FP16.merge(test_Zavic, on='documento', how='left').merge(test_Cleaver, on='documento', how='left').merge(test_Terman, on='documento', how='left').merge(test_Valores, on='documento', how='left')


Index(['fecha_de_aplicacion_x', 'pd_x', 'esten', 'pd1_x', 'esten1', 'pd2',
       'esten2', 'pd3', 'esten3', 'pd4', 'esten4', 'pd5', 'esten5', 'pd6',
       'esten6', 'pd7', 'esten7', 'pd8', 'esten8', 'pd9', 'esten9', 'pd10',
       'esten10', 'pd11', 'esten11', 'pd12', 'esten12', 'pd13', 'esten13',
       'pd14', 'esten14', 'pd15', 'esten15', 'pd16', 'esten16',
       'compatibilidad_x', 'documento', 'requi_x', 'fecha_de_aplicacion_y',
       'moral_mo', 'legalidad_le', 'indiferencia_in', 'corrupcion_co',
       'economico_ec', 'politico_po', 'social_so', 'religioso_re', 'requi_y',
       'fecha_de_aplicacion_x', 'combinacion', 'd', 'i', 's', 'c',
       'compatibilidad_y', 'd1', 'i1', 's1', 'c1', 'compatibilidad1', 'd2',
       'i2', 's2', 'c2', 'compatibilidad2', 'valida', 'requi_x',
       'fecha_de_aplicacion_y', 'informacion_i', 'juicio_ii',
       'vocabulario_iii', 'sintesis_iv', 'concentracion_v', 'analisis_vi',
       'abstraccion_vii', 'planeacion_viii', 'organizacion_ix', '